In [1]:
from socket import *
import sys
sys.path.append('Bin/')
import Lib
import re
from urllib import parse
from googletrans import Translator

translator = Translator()
libray = Lib.CppAssignment()
news = libray.Newspaper()

def k2u(text):
    url = text.replace('/', '|')
    print(url)
    url = parse.urlparse(f'http://www.exeam.org?examParam1=value1&examParam2={text}') 
    query = parse.parse_qs(url.query)
    data = parse.urlencode(query,  doseq=True)
    encording = data.replace('examParam1=value1&examParam2=', '')
    return encording

def cleanText(readData):
 
    #텍스트에 포함되어 있는 특수 문자 제거
 
    text = re.sub("[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]", '', readData)
 
    return text

In [11]:
from flask import Flask
import re

app = Flask(__name__)


@app.route('/')
def hello_world():
    return 'Hello, World!'


@app.route('/input/<data>')  # URL뒤에 <>을 이용해 가변 경로를 적는다
def hello_user(data):
    data = data.replace('%3A', ':')
    #data = data.replace('%3F', '?')
    url = data.replace('|', '/')
    print("URL :", url)
    try:
        data = news.Newspaper(url=url, language="ko")
        if len(data) == 0:
            data = news.Newspaper(url=url, language="en")
            if len(data) == 0:
                data = news.Newspaper(url=url, language="ja")
                if len(data) == 0:
                    data = news.Newspaper(url=url, language="zh")
                    print("SUMM :",data[0:50])
                    getSource = data[0:50]
                    source = libray.Papago(text=getSource).GetLangage()
                    print("SOURCE :", source)
                else:
                    print("SUMM :",data[0:50])
                    getSource = data[0:50]
                    source = libray.Papago(text=getSource).GetLangage()
                    print("SOURCE :", source)
            else:
                print("SUMM :",data[0:50])
                getSource = data[0:50]
                source = libray.Papago(text=getSource).GetLangage()
                print("SOURCE :", source)
        else:
            print("SUMM :",data[0:50])
            getSource = data[0:50]
            source = libray.Papago(text=getSource).GetLangage()
            print("SOURCE :", source)
    except:
        return f"[ERROR] CANT SUMMARY PLEASE CHECK THIS PAGE : {url}"
    
    
    try:
        if source != 'ko':
            print("[use google translator]")
            text = translator.translate(data, dest='ko').text
            print("ori :", data)
            print("tra :", text)
            sentence_data = libray.KoNLPy(text).sentence()
            data_pn_score = []
            obj = [0]
            for i in range(len(sentence_data)):
                if len(sentence_data[i]) < 21:
                    score = 0
                    tf = 0
                else:
                    _, score, tf = libray.model().text(sentence_data[i])
                    data_pn_score.append(libray.model().text(sentence_data[i])[1:])
                    #tf == 1 부정

                if tf == 1:
                    obj.append(-score)
                else:
                    obj.append(score)

            print("\n총 점수 : ",round(int(sum(obj)) / len(obj), 1))

            for i in range(len(sentence_data)):
                print(obj[i+1], sentence_data[i])

            if int(sum(obj)) < 0:
                tf = "Negative"
            elif(int(sum(obj)) == 0):
                tf = "neutral"
            else:
                tf = "Positive"


            a = f"""[{str(tf)} : {str(round(int(sum(obj)) / len(obj), 1))}] ************** {str(text)}"""
            return a

        else:
            print("in else")
            print(data)
            sentence_data = libray.KoNLPy(data).sentence()
            data_pn_score = []
            obj = [0]
            print(sentence_data)
            print("go for")
            for i in range(len(sentence_data)):
                print(len(sentence_data[i]) < 21)
                if len(sentence_data[i]) < 21:
                    score = 0
                    tf = 0
                else:
                    _, score, tf = libray.model().text(sentence_data[i])
                    
                    data_pn_score.append(libray.model().text(sentence_data[i])[1:])
                    #tf == 1 부정
                if tf == 1:
                    obj.append(-score)
                else:
                    obj.append(score)
            print("\n총 점수 : ",int(sum(obj)))

            for i in range(len(sentence_data)):
                print(obj[i+1], sentence_data[i])


            if int(sum(obj)) < 0:
                tf = "Negative"
            elif(int(sum(obj)) == 0):
                tf = "neutral"
            else:
                tf = "Positive"


            a = f"""[{str(tf)} : {str(round(int(sum(obj)) / len(obj), 1))}] ************** {str(data)}"""
            return a
    except:
        return data
    
if __name__ == "__main__":
    app.run()


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


URL : https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=102&oid=001&aid=0011293671
SUMM : 2019.12.23 xanadu@yna.co.kr 검찰, 이춘재 8차사건 "국과수 감정 조
SOURCE : ko
in else
2019.12.23 xanadu@yna.co.kr 검찰, 이춘재 8차사건 "국과수 감정 조작"…'재심 개시' 의견(수원=연합뉴스) 홍기원 기자 = 이진동 수원지검 2차장검사가 23일 오후 경기도 수원시 영통구 수원지방검찰청 브리핑실에서 열린 이춘재 연쇄살인 8차 사건 재심 관련 브리핑에서 취재진 질문에 답하고 있다.2019.12.23 xanadu@yna.co.kr 이춘재 연쇄살인 8차사건 재심 브리핑(수원=연합뉴스) 홍기원 기자 = 이진동 수원지검 2차장검사가 23일 오후 경기도 수원시 영통구 수원지방검찰청 브리핑실에서 열린 이춘재 연쇄살인 8차 사건 재심 관련 브리핑에서 취재진 질문에 답하고 있다.
['2019.12.23 xanadu@yna .co .kr 검찰, 이 춘재 8차 사건 " 국과수 감정 조작"…\' 재심 개시\' 의견( 수원= 연합 뉴스) 홍 기원 기자 = 이진 동 수원 지검 2차 장검 사가 23일 오후 경기도 수원시 영통 구 수원지방 검찰청 브리핑 실에서 열린 이 춘재 연쇄 살인 8차 사건 재심 관련 브리핑에서 취재진 질문에 답하고 있다.2019.12.23', 'xanadu@yna .co .kr 이 춘재 연쇄 살인 8차 사건 재심 브리핑( 수원= 연합 뉴스) 홍 기원 기자 = 이진 동 수원 지검 2차 장검 사가 23일 오후 경기도 수원시 영통 구 수원지방 검찰청 브리핑 실에서 열린 이 춘재 연쇄 살인 8차 사건 재심 관련 브리핑에서 취재진 질문에 답하고 있다.']
go for
False


127.0.0.1 - - [23/Dec/2019 19:00:26] "GET /input/https%3A%7C%7Cnews.naver.com%7Cmain%7Cread.nhn%3Fmode%3DLSD&mid=shm&sid1=102&oid=001&aid=0011293671 HTTP/1.1" 200 -


False

총 점수 :  200
100.0 2019.12.23 xanadu@yna .co .kr 검찰, 이 춘재 8차 사건 " 국과수 감정 조작"…' 재심 개시' 의견( 수원= 연합 뉴스) 홍 기원 기자 = 이진 동 수원 지검 2차 장검 사가 23일 오후 경기도 수원시 영통 구 수원지방 검찰청 브리핑 실에서 열린 이 춘재 연쇄 살인 8차 사건 재심 관련 브리핑에서 취재진 질문에 답하고 있다.2019.12.23
100.0 xanadu@yna .co .kr 이 춘재 연쇄 살인 8차 사건 재심 브리핑( 수원= 연합 뉴스) 홍 기원 기자 = 이진 동 수원 지검 2차 장검 사가 23일 오후 경기도 수원시 영통 구 수원지방 검찰청 브리핑 실에서 열린 이 춘재 연쇄 살인 8차 사건 재심 관련 브리핑에서 취재진 질문에 답하고 있다.
